### Problem setting
we consider a three stage multi-item news-vendor problem
1) First stage: agents require certain units of fix-investment from agents <br>
&emsp;variable:<br>
&emsp;&emsp; &emsp;$ x_1 $:vector - units of items to stock<br>
&emsp;coeffients:<br>
&emsp;&emsp; &emsp;$ c_1 $:vector - costs for stocking each unit<br>
            
2) Second stage: vendor buys news paper from agent, the available quantity range depends on the invesment <br>
&emsp;  variable:<br>
&emsp; &emsp; &emsp; $ x_2 $:vector - units of items to stock more<br>
&emsp; &emsp; &emsp; $ r_2 $:vector - remain quantity after retailing befor restocking<br>
&emsp; coeffients:<br>
&emsp; &emsp; &emsp; $ c_2 $:vector - costs for restocking <br>
&emsp; random variable:<br>
&emsp; &emsp; &emsp; $ d_2 $:vector - demands of items<br>
&emsp; &emsp; &emsp; $ p_2 $:vector - retail price of items

3) Third stage: vendor sells news paper to individuals<br>
&emsp;  random variables:<br>
&emsp; &emsp; &emsp; $ d_3 $:vector - demands of items <br>
&emsp; &emsp; &emsp; $ p_3 $:vector - retail price of items

#### Stage Scenario Data

In [1]:
# Let's say the problem

# with 5 items to stock and retail
n_item = 5

# with 4 scenario in second stage
S_scenario = 4
L_scenario = 2

# budge at first 
b_1 = 10

# additional budget 
b_2 = 5


import numpy as np
import numpy.random as random 

# costs
c_1 = random.randint(low=1,high = 10, size = n_item)
c_2 = random.randint(low=1,high = 10, size = n_item)



# price
p_2 = c_1 + random.randint(low=1,high = 10, size = n_item)
p_3 = c_1 + random.randint(low=1,high = 10, size = n_item)

# damands
d_2 = [random.randint(low=1,high = 10, size = n_item) for i in range(S_scenario)]
d_3_scenario_wrap = [[random.randint(low=1,high = 10, size = n_item) \
                          for j in range(L_scenario)]\
                                 for i in range(S_scenario)]
# Probabilities
d_2_prob = random.random_sample((S_scenario,))
d_2_prob = d_2_prob/np.sum(d_2_prob)
d_3_prob_wrap = [random.random_sample((L_scenario,)) for i in range(S_scenario) ]
d_3_prob_wrap = [ row/np.sum(row) for row in d_3_prob_wrap]

In [2]:
'''
Template of scenario data structure
:: dictionary
scenario['stage_n'] = {
    'dc_var':['x_1'],
    'rd_var':[],
    'cf':{'b_1':b_1,'c_1':c_1},
    'conditions':[],
    'conditions_val':[],
    'rd_prob':{}
}
'''

scenario = {'n_item':n_item,
            'S_scenario':S_scenario,
            'L_scenario':L_scenario
           }

scenario['stage_1'] = {
    'dc_var':['x_1'],
    'rd_var':[],
    'cf':{'b_1':b_1,'c_1':c_1},
    'condition':[],
    'conditions_val':[],
    'rd_prob':{}
}

scenario['stage_2'] = {
    'dc_var':['x_2','r_2'],
    'rd_var': ['d_2'],
    'cf':{'b_2':b_2,'c_2':c_2,'p_2':p_2},
    'condition': [],
    'conditions_val': [],
    'rd_prob':{
        0:(d_2,d_2_prob)
    }
}

scenario['stage_3'] = {
    'dc_var':[],
    'rd_var': ['d_3'],
    'cf':{'p_3':p_3},
    'condition': ['d_2'],
    'conditions_val': [d_2],
    'rd_prob':{
        i:(d_3_scenario_wrap[i],d_3_prob_wrap[i]) for i in range(len(d_3_scenario_wrap)) 
    }
}

In [3]:
scenario

{'n_item': 5,
 'S_scenario': 4,
 'L_scenario': 2,
 'stage_1': {'dc_var': ['x_1'],
  'rd_var': [],
  'cf': {'b_1': 10, 'c_1': array([8, 7, 4, 3, 2])},
  'condition': [],
  'conditions_val': [],
  'rd_prob': {}},
 'stage_2': {'dc_var': ['x_2', 'r_2'],
  'rd_var': ['d_2'],
  'cf': {'b_2': 5,
   'c_2': array([2, 1, 7, 1, 6]),
   'p_2': array([15, 13, 10,  7,  7])},
  'condition': [],
  'conditions_val': [],
  'rd_prob': {0: ([array([1, 4, 4, 7, 8]),
     array([4, 5, 7, 9, 5]),
     array([4, 5, 7, 3, 7]),
     array([5, 8, 4, 5, 5])],
    array([0.50416388, 0.36593363, 0.0744239 , 0.0554786 ]))}},
 'stage_3': {'dc_var': [],
  'rd_var': ['d_3'],
  'cf': {'p_3': array([17,  8, 12, 12,  6])},
  'condition': ['d_2'],
  'conditions_val': [[array([1, 4, 4, 7, 8]),
    array([4, 5, 7, 9, 5]),
    array([4, 5, 7, 3, 7]),
    array([5, 8, 4, 5, 5])]],
  'rd_prob': {0: ([array([5, 7, 5, 7, 4]), array([9, 1, 1, 3, 8])],
    array([0.81615324, 0.18384676])),
   1: ([array([7, 3, 7, 5, 6]), array([3, 

#### Benchmark of reward $Y$ for a given policy of $x_2$
Specifically,  $x_2(x_1, \xi^2_i)$<br>
$x_1$ is a real value vector and $x_2$ is function responding to scenarios and first decision<br>
No SSD contraints to consider

In [4]:
item_n = 5
### policy example ###
def x_2_policy(x_1, s):
    return x_1

# a simple policy to order the same as previous 

In [21]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict
from gurobipy import quicksum as qsum

def three_stage_problem_no_SSD(scenario, return_model=False):
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = {}
        for l in range(L_scenario):
            d_3[s][l] = scenario['stage_3']['rd_prob'][s][0][l]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = {}
        for l in range(L_scenario):
            prob_of_d_3[s][l] = scenario['stage_3']['rd_prob'][s][1][l]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    
    m = gp.Model(f'main')
    x_1 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 'x_1')
    x_2 = {}
    for s in range(S_scenario):
        x_2[s] = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = f'x_2_s={s}')
    
    # retail quantatity 
    # gurobi itself provide genral constraint to handle max or min functions  
    # which introduces extra binary variables into the model
    s_2 = {}
    for i in range(S_scenario):
        s_2[i] = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = f's_2_s={i}')
        for j in range(n_item):
            m.addConstr( s_2[i][j] == gp.min_(x_1[j], constant = d_2[i][j]))
    
    # budget constraint
    tot = qsum(x_1[i]*c_1[i] for i in range(n_item))
    m.addConstr(tot<=scenario['stage_1']['cf']['b_1'], name='budget_constr_stage_1')
    
    tot_2_l = {}
    tot_2_r = {}
    for i in range(S_scenario):
        tot_2_r[i] = qsum(p_2[j]*s_2[i][j] for j in range(n_item))
        tot_2_l[i] = qsum(x_2[i][j]*c_2[j] for j in range(n_item))
        m.addConstr(tot_2_l[i]<=tot_2_r[i]+scenario['stage_2']['cf']['b_2'], \
                  name='budget_constr_stage_2')
    
    # revenue
    revenue_2 = {}
    revenue_2_expected = qsum(tot_2_r[i]*prob_of_d_2[i] for i in range(S_scenario))
    
    revenue_3_conditional_expected = {}
    s_3 = {}
    for i in range(S_scenario):
        revenue_3 = {}
        excess_3 = {}
        s_3[i] = {}
        for l in range(L_scenario):
            revenue_3[l] = 0
            excess_3[l] = m.addVars(n_item, \
                                   vtype = GRB.CONTINUOUS, \
                                   name = f'excess_3_{i}_{l}')
            s_3[i][l] = m.addVars(n_item, \
                                   vtype = GRB.CONTINUOUS, \
                                   name = f's_3_{i}_{l}')
            for j in range(n_item):
                m.addConstr( excess_3[l][j] == x_2[i][j]+x_1[j]-s_2[i][j])
                m.addConstr( s_3[i][l][j] == gp.min_(excess_3[l][j],constant = d_3[i][l][j]))
                revenue_3[l] += p_3[j]*s_3[i][l][j]
        revenue_3_conditional_expected[i] = qsum(prob_of_d_3[i][l]*revenue_3[l]\
                                                 for l in range(L_scenario))
    revenue_3_expected = qsum(revenue_3_conditional_expected[i]*prob_of_d_2[i]\
                              for i in range(S_scenario))
    revenue =  revenue_2_expected + revenue_3_expected
    
    # since budget is constrained
    # maximize revenue is equal to maximize the profit
    obj = revenue
    m.setObjective(obj, GRB.MAXIMIZE)
    m.optimize()
    if return_model:
        return [_.X for _ in x_1.values()], \
                [[_.X for _ in x_2[k].values() ] for k in range(S_scenario)],\
    else:
        return obj.getValue()

In [22]:
three_stage_problem_no_SSD(scenario, return_model=True)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 45 rows, 125 columns and 205 nonzeros
Model fingerprint: 0xbc3fb293
Model has 60 general constraints
Variable types: 125 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-01, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+01]
  GenCon const rng [1e+00, 9e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 29 rows and 69 columns
Presolve time: 0.01s
Presolved: 16 rows, 56 columns, 91 nonzeros
Found heuristic solution: objective 182.9623535
Variable types: 54 continuous, 2 integer (2 binary)

Root relaxation: objective 2.393498e+02, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*  

([0.0, 0.0, 0.0, 0.0, 5.0],
 [[9.0, 7.0, 1.1428571428571428, 7.0, 0.0],
  [3.5, 7.0, 3.0, 5.0, 0.0],
  [5.0, 8.0, 2.857142857142857, 2.0, 0.0],
  [4.0, 9.0, 1.0, 7.0, 1.5]],
 10.0)

#### Benchmark

In [7]:
import numpy as np
import math

# a benchmark from the optimal policy

# function form of x_2
def x_2(x_1, d):
    n = len(x_1)
    rst = [0 for i in range(n)]
    for i in range(n):
        rst[i] = max(x_1[i]-d[i],d[i]) if x_1[i]>d[i] else d[i]
        
def x_2_i(x_1, d):
        return max(x_1-d,d) if x_1>d else d

def ssd_benchmark_from( x_1, x_2_i, scenario):
    
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = {}
        for l in range(L_scenario):
            d_3[s][l] = scenario['stage_3']['rd_prob'][s][0][l]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = {}
        for l in range(L_scenario):
            prob_of_d_3[s][l] = scenario['stage_3']['rd_prob'][s][1][l]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    
    
    benchmark = {}
    benchmark['stage_1'] = {
        'r': [-sum(x_1[i]*c_1[i] for i in range(n_item))],
        's': [1.0]
    }
    
    
    benchmark['stage_2'] = {
        'r': [sum(p_2[i]*min(d_2[s][i],x_1[i])-c_2[i]*x_2_i(x_1[i],d_2[s][i]) for i in range(n_item))\
                          for s in range(S_scenario)],
        's': prob_of_d_2
    }
    
    benchmark['stage_3'] = {
        'r': { 
                s:[sum(p_3[i]*min( d_3[s][l][i], x_2_i(x_1[i],d_2[s][i])) for i in range(n_item)) \
                   for l in range(L_scenario)] for s in range(S_scenario)
        },
        's': { 
                s:prob_of_d_3[s] for s in range(S_scenario)
        }
    }
    
    return benchmark

x_1 = [5,4,6,0,0]
benchmark = ssd_benchmark_from(x_1, x_2_i, scenario)
print(ssd_benchmark_from(x_1, x_2_i, scenario))

{'stage_1': {'r': [-92], 's': [1.0]}, 'stage_2': {'r': [12, 71, 65, 86], 's': array([0.50416388, 0.36593363, 0.0744239 , 0.0554786 ])}, 'stage_3': {'r': {0: [256, 172], 1: [266, 193], 2: [175, 210], 3: [217, 186]}, 's': {0: {0: 0.816153235677221, 1: 0.18384676432277905}, 1: {0: 0.1491173191115604, 1: 0.8508826808884397}, 2: {0: 0.7000929653843104, 1: 0.2999070346156896}, 3: {0: 0.36920868062198203, 1: 0.630791319378018}}}}


In [8]:
# utility tool functions
import decimal

def round2(c):
    c = decimal.Decimal(c)
    return float(round(c,2))


In [69]:
benchmark

{'stage_1': {'r': [-92], 's': [1.0]},
 'stage_2': {'r': [12, 71, 65, 86],
  's': array([0.50416388, 0.36593363, 0.0744239 , 0.0554786 ])},
 'stage_3': {'r': {0: [256, 172], 1: [266, 193], 2: [175, 210], 3: [217, 186]},
  's': {0: {0: 0.816153235677221, 1: 0.18384676432277905},
   1: {0: 0.1491173191115604, 1: 0.8508826808884397},
   2: {0: 0.7000929653843104, 1: 0.2999070346156896},
   3: {0: 0.36920868062198203, 1: 0.630791319378018}}}}

#### Multi-cut algorithm
we use the above result from non-SSD optimal policy as the benchmark

In [180]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def three_stage_multi_item_SSD_newsvendor(x_1_0, z_1_0, scenario, benchmark, max_itr = 100,\
                                         policy_return = False): 
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = scenario['stage_3']['rd_prob'][s][0]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = scenario['stage_3']['rd_prob'][s][1]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    b_2 = scenario['stage_2']['cf']['b_2']
    
    y_1 = benchmark['stage_1']['r'][0]
    
    y = {}
    for s in range(S_scenario):
        y[s] = {benchmark['stage_3']['r'][s][l]:benchmark['stage_3']['s'][s][l]\
                for l in range(L_scenario)}
        
    w = {}
    for s in range(S_scenario):
        w[s] = {}
        for y_3_j in y[s].keys():
            w[s][y_3_j] = sum(max(y_3_j-y_3_i,0)*prob for y_3_i,prob in y[s].items())
        
    theta = {}
    for s in range(S_scenario):
        theta[s] = benchmark['stage_2']['r'][s] + \
                        sum(  y_3_i*prob for y_3_i,prob in y[s].items())
    u = {}
    for s in range(S_scenario):
        u[s] =  sum( max(theta[s] -  theta[s_], 0)*prob_of_d_2[s_] for s_ in range(S_scenario))
        
    # initialize, step 0
    itr = 0
    
    event_n = 0
    event_list = []
    event_cut = {}
    obj_cut = []
    fs_cut = []
    
    x_1_ = x_1_0
    z_1_ = z_1_0
    
    v_ = {}
    for i in range(S_scenario):
        v_[i]= float('inf')
    
    # init the master problem
    # we set the gurobi model outside the iteration loop
    master = gp.Model('master problem')
    
    master.Params.LogToConsole = 0
    x_1 = master.addVars(n_item,lb=0,ub=10, vtype = GRB.CONTINUOUS, name = 'x_1')
    z_1 = master.addVar(lb = -float('inf'), ub = float('inf'),\
                            vtype = GRB.CONTINUOUS, name = 'z_1')
    
    # x_1 and z_1
    cost_tot_1 = qsum( c_1[i]*x_1[i] for i in range(n_item))
    master.addConstr(z_1+cost_tot_1<=0)
    
    # variable v for each second stage scenario
    v = {}
    for i in range(n_item):
        v[i] = master.addVar(lb = -float('inf'), ub = 1000,\
                               vtype = GRB.CONTINUOUS, name = f'v_s={i}')

    
    master_obj = z_1 + sum( v[i]*prob_of_d_2[i] for i in range(S_scenario) )
    master.setObjective(master_obj, GRB.MAXIMIZE)
    
    
    reward_second_stage = [float('inf') for s in range(S_scenario)]
    x_2_ = [[0 for i in range(n_item)] for s in range(S_scenario)]
    # cut inequalities are added into master during iterations
    while(itr<max_itr):
        
        all_solvable_flag = True
        for s in range(S_scenario):
            rst_obj = reward_problem_of_first_stage_ssd(x_1_, z_1_, s, \
                                                  d_2[s],
                                                  d_3[s], prob_of_d_3[s], \
                                                  benchmark,\
                                                  w[s], \
                                                  b_2, p_2, p_3, c_2)
            
            if rst_obj is not None:
                # try objective cuts
                x_2_[s], _, reward_second_stage[s], pi_1_,pi_2_ = rst_obj
                # print(s, rst_obj)
                tem_obj_cut = master.addConstr( v[s] <=reward_second_stage[s] +\
                                               qsum(pi_1_[i]*(x_1[i]-x_1_[i])\
                                                                for i in range(n_item))\
                                                         - pi_2_*(z_1 - z_1_),\
                                              name= f'obj_cut_{itr}_{s}')
                obj_cut.append(tem_obj_cut)
            else:
                # try feasibility cuts
                rst_fs = feasibility_problem_of_first_stage_ssd(x_1_, z_1_, s, \
                                                  d_2[s],
                                                  d_3[s], prob_of_d_3[s], \
                                                  benchmark,\
                                                  w[s], \
                                                  b_2, p_2, p_3, c_2)
                print(rst_fs)
                if rst_fs=='optimal': 
                    continue
                elif rst_fs=='beyond max_itr':
                    raise RuntimeError(rst_fs + f' in {itr} scenario = {s}')
                all_solvable_flag = False
                _, _, fs_obj, pi_1_,pi_2_ = rst_fs
                tem_fs_cut = master.addConstr(  0 >= fs_obj +\
                                             qsum(pi_1_[i]*(x_1[i]-x_1_[i])\
                                                                for i in range(n_item))\
                                                     - pi_2_*(z_1 - z_1_), \
                                             name= f'fs_cut_{itr}_{s}')
                fs_cut.append(tem_fs_cut)
                
        # update (add) event cuts to master problem
        event_n_ = event_n 
        if all_solvable_flag:
            A = {}
            tem_sup = {}
            for j in range(S_scenario):
                theta_j = theta[j]
                A[j] = [i for i in range(S_scenario) if v_[i] + z_1_-y_1<=theta[i]]

                tem_sup[j] = sum((theta[j] - v_[i]-z_1_+y_1)*prob_of_d_2[i]\
                                       for i in A[j]) - u[j]
                
            tem_sup_ = max(sup for key,sup in tem_sup.items())
            if tem_sup_ > 0:
                tem_sup_ = tem_sup_/2
                for s in range(S_scenario):
                    if tem_sup[s]>=tem_sup_:
                        # add new constraints
                        tot = 0
                        for j in A[s]:
                            tem_lhs = theta[s] -v[j] -z_1+y_1
                            tot += prob_of_d_2[j]*tem_lhs
                        master.addConstr(tot<=u[s],
                                        name= f'event_cut_{itr}_{s}')
                        event_n +=1

        # print('abs',[(v_[s], reward_second_stage[s]) for s in range(S_scenario)])
        if event_n==event_n_ and all_solvable_flag\
                and all(abs(v_[s]- reward_second_stage[s])<0.1\
                                     for s in range(S_scenario)):
            break
        
        
        
        # master solution
        master.optimize()
        # print(master.display())
        # print(master.Status)
        if master.Status==4:
            return 'unbounded'
        
        # update values of variables
        x_1_ = [x_1[i].X for i in range(n_item)] 
        z_1_=round2(z_1.X)
        for s in range(S_scenario):
            v_[s]= round2(v[s].X)
        # print(itr,'x_1: ', x_1_,'z_1: ', z_1_)
            
        # increase k by 1
        itr+=1
    print(itr,'iterations')
    if (itr>=max_itr):
        return "terminated: max iteration"
    elif policy_return:
        return x_1_,x_2_
    else:
        return x_1_,z_1_

# testing
x_1_0 = [0.0, 0.0, 0.0, 0.0, 5.0]
z_1_0 = -10
three_stage_multi_item_SSD_newsvendor(x_1_0 = x_1_0, z_1_0 =z_1_0, scenario=scenario, \
                                benchmark=benchmark, max_itr = 20)

2 iterations


([0.0, 0.0, 0.0, 0.0, 10.0], -20.0)

##### Solver for Subproblems

In [143]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def reward_problem_of_first_stage_ssd(x_1:list, z_1:float, s:int, \
                                      d_2:list,\
                                      d_3:dict,prob_of_d_3:list, 
                                      benchmark:dict,\
                                      w, \
                                      b_2, p_2, p_3, c_2):
    L_scenario = len(prob_of_d_3)
    n_item = len(p_3)
    
    itr = 0
    max_itr = 100
    
    m = gp.Model('reward_problem_two_stage_multi_item_SSD')
    m.Params.LogToConsole = 0
    
    s_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 's_2')
    x_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 'x_2')
    z_2 = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'z_2')
    sigma = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'sigma')
    
    pi_1 = m.addConstrs((s_2[j] <= x_2[j] for j in range(n_item)))
    m.addConstrs((s_2[j] <= d_2[j] for j in range(n_item)))
        
    f_3 = {}
    for i in range(L_scenario):
        f_3_tem = m.addVars(n_item,vtype = GRB.CONTINUOUS, name = f'f_3_l={i}')
        m.addConstrs((f_3_tem[j] <= p_3[j]*d_3[i][j] for j in range(n_item)))
        m.addConstrs((f_3_tem[j] <= p_3[j]*x_2[j] for j in range(n_item)))
        f_3[i] = qsum( f_3_tem[j] for j in range(n_item))
    obj = z_2+ qsum(f_3[l]*prob_of_d_3[l] for l in range(L_scenario))
    m.setObjective(obj,GRB.MAXIMIZE)
    
    tot_cost_2 = qsum( c_2[i]*x_2[i] for i in range(n_item))
    tot_revenue_2 = sum(s_2[i]*p_2[i] for i in range(n_item))
    m.addConstr( tot_cost_2 <=\
                          tot_revenue_2 + b_2, name='budget_limit' )
    m.addConstr( z_2<= tot_revenue_2-tot_cost_2, 'z_2<=f(x)')
    
    pi_2 = m.addConstr(sigma ==z_1 + z_2 - \
                       benchmark['stage_1']['r'][0] - \
                       benchmark['stage_2']['r'][s] , name = 'sigma')
    
    # union hte same rewards in benchmark

    f_3_ = {}
    for i in range(L_scenario):
        f_3_[i] = - float('inf')
    x_2_ = [0 for i in range(n_item)]
    z_2_ = 0
        
    while( itr< max_itr):
        m.optimize()
        # print(m.display())
        if m.Status!=2:
            print(m, 'status:',m.Status)
            return  None
        for i in range(n_item):
            x_2_[i] = x_2[i].X
        z_2_ = z_2.X
        sigma_ = sigma.X
        for i in range(L_scenario):
            f_3_[i] = f_3[i].getValue()
        obj_ = obj.getValue()
        
        all_sat_flag = True
        for y_3_j,w_j in w.items():
            if sum(max(y_3_j-sigma_-f_3_[l],0)*prob_of_d_3[l] for l in range(L_scenario))>w_j:
                all_sat_flag = False
                A = [l for l in range(L_scenario) if y_3_j>sigma_+f_3_[l]]
                a = qsum( (y_3_j-sigma-f_3[d])*prob_of_d_3[d] for d in A)
                m.addConstr( a<=w_j, name= f'event_cut_{itr}_{y_3_j}_{w_j}')

        if itr!=0 and all_sat_flag:
            pi_1_=[pi_1[i].Pi for i in range(n_item)]
            pi_2_=pi_2.Pi
            # print(itr)
            return x_2_,z_2_,obj_,pi_1_,pi_2_
        itr +=1
    return None

In [172]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def feasibility_problem_of_first_stage_ssd(x_1:list, z_1:float, s:int, \
                                      d_2:list,\
                                      d_3:dict,prob_of_d_3:dict, 
                                      benchmark:dict,\
                                      w, \
                                      b_2, p_2, p_3, c_2):
    L_scenario = len(prob_of_d_3)
    n_item = len(p_3)
    
    itr = 0
    max_itr = 100
    
    m = gp.Model('feasiblity_two_stage_SSD')
    m.Params.LogToConsole = 0
    
    s_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 's_2')
    x_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 'x_2')
    z_2 = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'z_2')
    sigma = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'sigma')
    u_1 =m.addVars(n_item,lb = -float('inf'), ub = float('inf'), \
                       vtype = GRB.CONTINUOUS, name = 'u_1')
    u_2 = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'u_2')
    u_1_abs = m.addVars(n_item,vtype = GRB.CONTINUOUS, name = 'u_1_abs')
    u_2_abs = m.addVar(vtype = GRB.CONTINUOUS, name = 'u_2_abs')
    
    m.addConstr( (u_1_abs[i] >= u_1[i] for i in range(n_item)))
    m.addConstr( (u_1_abs[i] >= -u_1[i] for i in range(n_item)))
    m.addConstr( u_2_abs >= u_2)
    m.addConstr( u_2_abs >= -u_2)
    
                
    
    f_3 = {}
    for i in range(L_scenario):
        f_3_tem = m.addVars(n_item,vtype = GRB.CONTINUOUS, name = f'f_3_l={i}')
        m.addConstrs((f_3_tem[j] <= p_3[j]*d_3[i][j] for j in range(n_item)))
        m.addConstrs((f_3_tem[j] <= p_3[j]*x_2[j] for j in range(n_item)))
        f_3[i] = qsum( f_3_tem[j] for j in range(n_item))
    
    obj = qsum(u_1_abs[i] for i in range(n_item)) + u_2_abs
    m.setObjective(obj,GRB.MINIMIZE)
    
    pi_1 = m.addConstrs((s_2[j]+u_1[j] <= x_2[j] for j in range(n_item)))
    m.addConstrs((s_2[j] <= d_2[j] for j in range(n_item)))
    tot_cost_2 = qsum( c_2[i]*x_2[i] for i in range(n_item))
    tot_revenue_2 = sum(s_2[i]*p_2[i] for i in range(n_item))
    m.addConstr( tot_cost_2+u_1 <=\
                          tot_revenue_2 + b_2, name='budget_limit' )
    m.addConstr( z_2+ u_2 <= tot_revenue_2-tot_cost_2, 'z_2<=f(x)')
    
    pi_2 = m.addConstr(sigma ==z_1 + z_2 - \
                       benchmark['stage_1']['r'][0] - \
                       benchmark['stage_2']['r'][s] , name = 'sigma')
    
    # union hte same rewards in benchmark

    f_3_ = {}
    for i in range(L_scenario):
        f_3_[i] = - float('inf')
    x_2_ = [0 for i in range(n_item)]
    z_2_ = 0
        
    while( itr< max_itr):
        m.optimize()
        # print(m.display())
        if m.Status!=2:
            print(m, 'status:',m.Status)
            return  None
        for i in range(n_item):
            x_2_[i] = x_2[i].X
        z_2_ = z_2.X
        sigma_ = sigma.X
        for i in range(L_scenario):
            f_3_[i] = f_3[i].getValue()
        obj_ = obj.getValue()
        
        all_sat_flag = True
        for y_3_j,w_j in w.items():
            if sum(max(y_3_j-sigma_-f_3_[l],0)*prob_of_d_3[l] for l in range(L_scenario))>w_j+0.01:
                all_sat_flag = False
                A = [l for l in range(L_scenario) if y_3_j>sigma_+f_3_[l]]
                a = qsum( (y_3_j-sigma-f_3[d])*prob_of_d_3[d] for d in A)
                m.addConstr( a<=w_j, name= f'event_cut_{itr}_{y_3_j}_{w_j}')

        if itr!=0 and all_sat_flag:
            pi_1_=pi_1.Pi
            pi_2_=pi_2.Pi
            # print(f'iteration times: {itr+1}')
            return x_2_,z_2_, obj_,pi_1_,pi_2_
        itr +=1
    
    return 'beyond max_itr'

### testing

In [28]:
S_scenario = scenario['S_scenario']
L_scenario = scenario['L_scenario']
y = {}
for s in range(S_scenario):
    y[s] = {benchmark['stage_3']['r'][s][l]:benchmark['stage_3']['s'][s][l] \
            for l in range(L_scenario)}
w = {}
for s in range(S_scenario):
    w[s] = {}
    for y_3_j in y[s].keys():
        w[s][y_3_j] = sum(max(y_3_j-y_3_i,0)*prob for y_3_i,prob in y[s].items())

theta = {}
for s in range(S_scenario):
    theta[s] = benchmark['stage_2']['r'][s] + \
                    sum(  y_3_i*prob for y_3_i,prob in y[s].items())

In [135]:
x_1 = [0.0, 0.0, 0.0, 10.0, 2.476834722487933]
z_1 = -34.95
s = 0
prob_of_d_3 = scenario['stage_3']['rd_prob'][s][1]
d_2 = scenario['stage_2']['rd_prob'][0][0]
b_2 = scenario['stage_2']['cf']['b_2']
p_2 = scenario['stage_2']['cf']['p_2']
c_2 = scenario['stage_2']['cf']['c_2']
p_3 = scenario['stage_3']['cf']['p_3']
d_3 = scenario['stage_3']['rd_prob'][s][0]
reward_problem_of_first_stage_ssd(x_1, z_1, s, \
                                  d_2[s],
                                  d_3,prob_of_d_3, \
                                  benchmark,\
                                      w[s], \
                                      b_2, p_2, p_3, c_2)

([9.0, 7.0, 5.0, 7.0, 0.7229738429025877], -5.0, 270.3654889688843, -0.0, -0.0)

In [108]:
S_scenario = scenario['S_scenario']
L_scenario = scenario['L_scenario']
y = {}
for s in range(S_scenario):
    y[s] = {benchmark['stage_3']['r'][s][l]:benchmark['stage_3']['s'][s][l] \
            for l in range(L_scenario)}
w = {}
for s in range(S_scenario):
    w[s] = {}
    for y_3_j in y[s].keys():
        w[s][y_3_j] = sum(max(y_3_j-y_3_i,0)*prob for y_3_i,prob in y[s].items())

theta = {}
for s in range(S_scenario):
    theta[s] = benchmark['stage_2']['r'][s] + \
                    sum(  y_3_i*prob for y_3_i,prob in y[s].items())

x_1 = [0.0, 0.0, 0.0, 0.0, 5.0]
z_1 = 0
s = 2
prob_of_d_3 = scenario['stage_3']['rd_prob'][s][1]
b_2 = scenario['stage_2']['cf']['b_2']
p_2 = scenario['stage_2']['cf']['p_2']
c_2 = scenario['stage_2']['cf']['c_2']
p_3 = scenario['stage_3']['cf']['p_3']
d_3 = scenario['stage_3']['rd_prob'][s][0]
testing_feasibility_problem_of_first_stage_ssd(x_1, z_1, s, \
                                  d_3,prob_of_d_3, \
                                  benchmark,\
                                      w[s], \
                                      b_2, p_2, p_3, c_2)

0 i's: [0, 1]
0 i's: [0, 1]
1 i's: [0]
1 i's: [0]


([5.0, 2.9852941176470553, 3.5735294117647065, 2.0, 0.0],
 -5.0,
 19.117647058823543,
 -0.41176470588235287,
 -1.0)

### Policy evaluation

In [160]:
def evaluation_policy(x_1, x_2, scenario):
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = scenario['stage_3']['rd_prob'][s][0]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = scenario['stage_3']['rd_prob'][s][1]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    b_2 = scenario['stage_2']['cf']['b_2']
    
    rst = {}
    rst['stage_1']={
        'revenue': 0,
        'cost':-sum(c_1[j]*x_1[j] for j in range(n_item)),
        'prob': 1.0
    }
    rst['stage_2'] = {}
    rst['stage_3'] = {}
    
    for s in range(S_scenario):
        rst['stage_2'][s]={
            'revenue': sum(p_2[j]*min(x_1[j], d_2[s][j]) for j in range(n_item)),
            'cost':-sum(c_2[j]*x_2[s][j] for j in range(n_item)),
            'prob':prob_of_d_2[s]
        }
        rst['stage_3'][s] = {}
        for l in range(L_scenario):
            rst['stage_3'][s][l] = {
                'revenue': sum(p_3[j]*min(x_2[s][j], d_3[s][l][j]) for j in range(n_item)),
                'cost':0,
                'prob':prob_of_d_3[s][l]
            }
    return rst
###### testing
x_1 = [0.0, 0.0, 0.0, 0.0, 5.0]
x_2 = [[9.0, 7.0, 1.1428571428571428, 7.0, 0.0],
  [3.5, 7.0, 3.0, 5.0, 0.0],
  [5.0, 8.0, 2.857142857142857, 2.0, 0.0],
  [4.0, 9.0, 1.0, 7.0, 1.5]]
evaluation_policy(x_1,x_2,scenario)

{'stage_1': {'revenue': 0, 'cost': -10.0, 'prob': 1.0},
 'stage_2': {0: {'revenue': 35.0, 'cost': -40.0, 'prob': 0.5041638806771759},
  1: {'revenue': 35.0, 'cost': -40.0, 'prob': 0.3659336256558766},
  2: {'revenue': 35.0, 'cost': -40.0, 'prob': 0.07442389719609228},
  3: {'revenue': 35.0, 'cost': -40.0, 'prob': 0.055478596470855165}},
 'stage_3': {0: {0: {'revenue': 238.71428571428572,
    'cost': 0,
    'prob': 0.816153235677221},
   1: {'revenue': 209.0, 'cost': 0, 'prob': 0.18384676432277905}},
  1: {0: {'revenue': 179.5, 'cost': 0, 'prob': 0.1491173191115604},
   1: {'revenue': 179.0, 'cost': 0, 'prob': 0.8508826808884397}},
  2: {0: {'revenue': 125.28571428571428,
    'cost': 0,
    'prob': 0.7000929653843104},
   1: {'revenue': 207.28571428571428, 'cost': 0, 'prob': 0.2999070346156896}},
  3: {0: {'revenue': 196.0, 'cost': 0, 'prob': 0.36920868062198203},
   1: {'revenue': 173.0, 'cost': 0, 'prob': 0.630791319378018}}}}

In [162]:
def profit_distribution(evaluation_rst):
    profit = {}
    profit_1 = evaluation_rst['stage_1']['revenue'] + evaluation_rst['stage_1']['cost']
    profit_2 = {}
    profit_3 = {}
    for i,dict_2 in evaluation_rst['stage_2'].items():
        profit_2[i] = dict_2['revenue'] + dict_2['cost']
        for j,dict_3 in evaluation_rst['stage_3'][i].items():
            profit_3[(i,j)] = dict_3['revenue'] + dict_2['cost']
    return profit_3
###### testing
evaluation_rst = evaluation_policy(x_1,x_2,scenario)
profit_distribution(evaluation_rst)

{(0, 0): 198.71428571428572,
 (0, 1): 169.0,
 (1, 0): 139.5,
 (1, 1): 139.0,
 (2, 0): 85.28571428571428,
 (2, 1): 167.28571428571428,
 (3, 0): 156.0,
 (3, 1): 133.0}

In [163]:
###### evaluation reagrding SSD 
x_1 = [0.0, 0.0, 0.0, 0.0, 10.0]
x_2 = [[9.0, 7.0, 5.0, 7.0, 8.0],
 [7.0, 7.0, 7.0, 9.0, 6.0],
 [5.0, 8.0, 7.0, 3.0, 7.0],
 [5.0, 9.0, 4.0, 7.0, 5.0]
]
evaluation_rst = evaluation_policy(x_1,x_2,scenario)
profit_distribution(evaluation_rst)

{(0, 0): 194.0,
 (0, 1): 142.0,
 (1, 0): 208.0,
 (1, 1): 100.0,
 (2, 0): 63.0,
 (2, 1): 139.0,
 (3, 0): 157.0,
 (3, 1): 110.0}

In [170]:
###### evaluation reagrding dummy policy
x_1 = [5,4,6,0,0]
def x_2_i(x_1, d):
        return max(x_1-d,d) if x_1>d else d
x_2 = [[x_2_i(x_1[j],d_2[j]) for j in range(n_item)] \
       for d_2 in scenario['stage_2']['rd_prob'][0][0] ]
evaluation_rst = evaluation_policy(x_1,x_2,scenario)
profit_distribution(evaluation_rst)

{(0, 0): 161,
 (0, 1): 77,
 (1, 0): 165,
 (1, 1): 92,
 (2, 0): 68,
 (2, 1): 103,
 (3, 0): 136,
 (3, 1): 105}

In [183]:
x_1 = [0.0, 0.0, 0.0, 0.0, 5.0]
x_2 = [[9.0, 7.0, 1.1428571428571428, 7.0, 0.0],
  [3.5, 7.0, 3.0, 5.0, 0.0],
  [5.0, 8.0, 2.857142857142857, 2.0, 0.0],
  [4.0, 9.0, 1.0, 7.0, 1.5]]
def ssd_benchmark_from( x_1, x_2, scenario):
    
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = {}
        for l in range(L_scenario):
            d_3[s][l] = scenario['stage_3']['rd_prob'][s][0][l]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = {}
        for l in range(L_scenario):
            prob_of_d_3[s][l] = scenario['stage_3']['rd_prob'][s][1][l]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    
    
    benchmark = {}
    benchmark['stage_1'] = {
        'r': [-sum(x_1[i]*c_1[i] for i in range(n_item))],
        's': [1.0]
    }
    
    
    benchmark['stage_2'] = {
        'r': [sum(p_2[i]*min(d_2[s][i],x_1[i])-c_2[i]*x_2[s][i] for i in range(n_item))\
                          for s in range(S_scenario)],
        's': prob_of_d_2
    }
    
    benchmark['stage_3'] = {
        'r': { 
                s:[sum(p_3[i]*min( d_3[s][l][i], x_2[s][i]) for i in range(n_item)) \
                   for l in range(L_scenario)] for s in range(S_scenario)
        },
        's': { 
                s:prob_of_d_3[s] for s in range(S_scenario)
        }
    }
    
    return benchmark
non_ssd_benchmark = ssd_benchmark_from( x_1, x_2, scenario)
x_1,x_2 = three_stage_multi_item_SSD_newsvendor(x_1_0 = x_1_0, z_1_0 =z_1_0, scenario=scenario,\
                                benchmark=non_ssd_benchmark, max_itr = 20,\
                                policy_return =  True)
evaluation_rst = evaluation_policy(x_1,x_2,scenario)
profit_distribution(evaluation_rst)

2 iterations


{(0, 0): 194.0,
 (0, 1): 142.0,
 (1, 0): 208.0,
 (1, 1): 100.0,
 (2, 0): 63.0,
 (2, 1): 139.0,
 (3, 0): 157.0,
 (3, 1): 110.0}